In [1]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')

dt = train['dt'].values

class MyData(Dataset):
    def __init__(self, traindata, numstep, length):
        self.serial_number = traindata['serial_number'].value_counts()
        self.value = traindata.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.input = []
        self.label = []
        for diskname in self.serial_number.index.sort_values():
            traindata_name = traindata[traindata.serial_number == diskname]
            self.datalabel = traindata_name.loc[:, 'label'].values.tolist()
            j = 0
            while(j + numstep <= len(traindata_name)):
                self.input.append(torch.Tensor(self.datas[j:j + numstep]))
                if(torch.Tensor(self.datalabel[j:j + numstep]).sum() != 0):
                    self.label.append(torch.ones(1))
                else:
                    self.label.append(torch.zeros(1))
                j = j + 7  
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]
train_data = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
train_set = MyData(train_data, 7, 6)
train_iter = DataLoader(MyData(train_data, 7, 6), batch_size = 10)


In [34]:
num = 0
for X, y in train_set:
    num = num + y
total = len(train_set)

In [35]:
total

1243

In [9]:
class_sample_count = np.array([total - num, num])
class_sample_count

array([974., 269.], dtype=float32)

In [11]:
weight = 1. / class_sample_count
weight

array([0.00102669, 0.00371747], dtype=float32)

In [29]:
sample_list = []
for x, y in train_set:
    sample_list.append(weight[int(y)])


In [33]:
sample_weight = torch.FloatTensor(sample_list)
sample_weight.shape

torch.Size([1243])

In [37]:
sampler = WeightedRandomSampler(sample_weight, total)
sampler

In [38]:
train_iter = DataLoader(train_set, batch_size=10)  